# BiLSTM for Sentiment Classification

This Conv Net performs sentiment analysis on the Google toxicity dataset review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text
from keras.models import Model, Input
from keras.layers import Input, concatenate
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Embedding, Conv1D, SpatialDropout1D, GlobalMaxPool1D, LSTM
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import numpy as np

import pandas as pd

import os

import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/bi-lstm'

n_classes = 6

epochs = 3
batch_size = 64
test_split=.1

n_dim = 32
n_unique_words = 20000
max_review_length = 300
pad_type = trunc_type = 'pre'

n_bilstm_1 = 8
n_bilstm_2 = 16
drop_bilstm = 0.25

#### Load Data

In [ ]:
train_df = pd.read_csv('kaggle/datasets/toxicity/train.csv')
test_df = pd.read_csv('kaggle/datasets/toxicity/test.csv')

#### Preprocess Data

In [ ]:
test_df.shape

In [ ]:
train_sentences_series = train_df['comment_text'].fillna("_").values
test_sentences_series = test_df['comment_text'].fillna("_").values

# Tokeninze the Training data
tokenizer = text.Tokenizer(num_words=n_unique_words)
tokenizer.fit_on_texts(list(train_sentences_series))
train_tokenized_sentences = tokenizer.texts_to_sequences(train_sentences_series)

# Tokeninze the Test data
test_tokenized_sentences = tokenizer.texts_to_sequences(test_sentences_series)

# toxic,severe_toxic,obscene,threat,insult,identity_hate
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train_df[classes].values

X_train = pad_sequences(train_tokenized_sentences, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_test_sub = pad_sequences(test_tokenized_sentences, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_split)

#### Custom Activiation Function

In [ ]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class ReLUs(Activation):
    
    def __init__(self, activation, **kwargs):
        super(ReLUs, self).__init__(activation, **kwargs)
        self.__name__ = 'ReLU_s'

def relus(Z):
    e_param = 1.8
    pi = K.variable((3.14))
    m = e_param + (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'ReLU_s': ReLUs(relus)})

#### Design Deep Net Architecture

In [ ]:
input_layer = Input(shape=(max_review_length,), dtype='int16', name='input')
embedding_layer = Embedding(n_unique_words, n_dim, input_length=max_review_length,
                            name='embedding_1')(input_layer)

bi_lstm_1 = Bidirectional(LSTM(n_bilstm_1, dropout=drop_bilstm, return_sequences=True,
                               name='bi_lstm_1'))(embedding_layer)
bi_lstm_2 = Bidirectional(LSTM(n_bilstm_2, dropout=drop_bilstm, return_sequences=True,
                               name='bi_lstm_2'))(embedding_layer)

concat = concatenate([bi_lstm_1, bi_lstm_2])

densor = Dense(64, activation="relu")(concat)

flat = Flatten()(densor)

output = Dense(n_classes, activation='softmax', name='output')(flat)

model = Model(input_layer, output)

In [ ]:
model.summary()

#### Configure the Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelCheckpoint = ModelCheckpoint(filepath=output_dir+'/weights-bi-lstm-toxicity_new.hdf5', save_best_only=True, mode='min')
earlyStopping = EarlyStopping(mode='min', patience=1)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_valid, y_valid), callbacks=[modelCheckpoint, earlyStopping])

#### Evaluate

In [ ]:
#model.load_weights(output_dir+'/weights-multicnn-toxicity.hdf5')
model = keras.models.load_model(output_dir + '/weights-bi-lstm-toxicity_new.hdf5')

In [ ]:
y_hat = model.predict(X_test_sub)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
np.random.shuffle(y_hat)
pct_auc = roc_auc_score(y_valid, y_hat[0:31915]) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)

In [ ]:
y_hat[0]

In [ ]:
sample_submission = pd.read_csv("kaggle/datasets/toxicity/sample_submission.csv")

sample_submission.shape

sample_submission[classes] = y_hat
sample_submission.to_csv("kaggle/datasets/toxicity/submission_bi-lstm_relus.csv", index=False)